In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [87]:
# load data

global training___data, validation_data, training___price, validation_price
def load_data():
    global training___data, validation_data, training___price, validation_price
    data = pd.read_csv("./train.csv")
    training___data, validation_data, training___price, validation_price = train_test_split(data, data.SalePrice, random_state = 0)

load_data()
training___data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1292,1293,70,RM,60.0,6600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2009,WD,Normal,107500
1018,1019,80,RL,NaN,10784,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,160000
1213,1214,80,RL,NaN,10246,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,145000
1430,1431,60,RL,60.0,21930,Pave,NaN,IR3,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Normal,192140
810,811,20,RL,78.0,10140,Pave,NaN,Reg,Lvl,AllPub,...,648,Fa,GdPrv,NaN,0,1,2006,WD,Normal,181000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,764,60,RL,82.0,9430,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,337000
835,836,20,RL,60.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2010,WD,Normal,128000
1216,1217,90,RM,68.0,8930,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,112000
559,560,120,RL,NaN,3196,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,234000


In [88]:
# define possible column transformations

def drop(column_name, data_set, *_):
    data_set.drop(column_name, axis=1, inplace=True)
    return data_set

def drop_row(column_name, data_set, condition):
    return data_set[~data_set[column_name].apply(condition)]

def one_hot(column_name, data_set, options):
    keys = options if options else data_set[column_name].unique()
    for key in keys:
        data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if x == key else 0)
    return drop(column_name, data_set)

def MSZoning(column_name, data_set, *_):
    options = dict(
        # A="Agriculture",
        C="Commercial",
        FV="Floating Village Residential",
        # I="Industrial",
        RH="Residential High Density",
        RL="Residential Low Density",
        # RP="Residential Low Density Park",
        RM="Residential Medium Density",
    )
    for key, name in options.items():
        if key == "C":
            continue
        data_set[f"zoned_{name}"] = data_set[column_name].apply(lambda x: 1 if x in [key, "C"] else 0)
    return drop(column_name, data_set)

is_nan = lambda x: x != x
all_unique_values = lambda column_name: set(training___data[column_name].unique()) | set(validation_data[column_name].unique())

In [89]:
# specify each column transformation

transformations = {
    "MSSubClass": (drop, None),
    "MSZoning": (MSZoning, None),
    "LotFrontage": (drop_row, is_nan),
    "Street": (one_hot, all_unique_values("Street")),
}

In [90]:
# apply column transformations to data
load_data()
for column_name, (transform, option) in transformations.items():
    print("transforming", column_name)
    training___data = transform(column_name, training___data, option)
    validation_data = transform(column_name, validation_data, option)

validation_data

transforming MSSubClass
transforming MSZoning
transforming LotFrontage
transforming Street


,Id,LotFrontage,LotArea,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,YrSold,SaleType,SaleCondition,SalePrice,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,Street_Grvl,Street_Pave
491,492,79.0,9490,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,2006,WD,Normal,133000,0,0,1,0,0,1
279,280,83.0,10005,NaN,Reg,Lvl,AllPub,Inside,Gtl,ClearCr,...,2008,WD,Normal,192000,0,0,1,0,0,1
655,656,21.0,1680,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrDale,...,2010,WD,Family,88000,0,0,0,1,0,1
1013,1014,60.0,7200,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,...,2009,WD,Normal,85000,0,0,0,1,0,1
1403,1404,49.0,15256,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Somerst,...,2007,WD,Normal,282922,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,584,75.0,13500,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,...,2008,WD,Normal,325000,0,0,0,1,0,1
1245,1246,78.0,12090,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,2007,WD,Abnorml,178000,0,0,1,0,0,1
1390,1391,70.0,9100,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,2006,WD,Normal,235000,0,0,1,0,0,1
1375,1376,89.0,10991,NaN,IR1,HLS,AllPub,Inside,Gtl,Timber,...,2007,New,Partial,239000,0,0,1,0,0,1


In [91]:
training___data.describe()
validation_data.describe()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,MiscVal,MoSold,YrSold,SalePrice,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,Street_Grvl,Street_Pave
count,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,303.000000,305.000000,305.000000,...,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000
mean,710.842623,70.888525,9744.954098,6.193443,5.639344,1971.455738,1985.750820,100.633663,439.570492,46.881967,...,14.426230,6.462295,2007.783607,182098.911475,0.055738,0.003279,0.744262,0.193443,0.003279,0.996721
std,422.048717,27.455119,5841.765318,1.486327,1.121453,31.435379,21.227547,178.481456,532.480463,170.622484,...,133.728473,2.678998,1.297414,86517.393336,0.229792,0.057260,0.436992,0.395646,0.057260,0.057260
min,2.000000,21.000000,1477.000000,3.000000,3.000000,1882.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,2006.000000,67000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,363.000000,60.000000,7200.000000,5.000000,5.000000,1950.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,5.000000,2007.000000,125500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,668.000000,69.000000,9084.000000,6.000000,5.000000,1973.000000,1995.000000,0.000000,360.000000,0.000000,...,0.000000,6.000000,2008.000000,160000.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1074.000000,80.000000,11310.000000,7.000000,6.000000,2003.000000,2005.000000,141.500000,686.000000,0.000000,...,0.000000,8.000000,2009.000000,208900.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1455.000000,313.000000,63887.000000,10.000000,9.000000,2009.000000,2009.000000,1031.000000,5644.000000,1474.000000,...,2000.000000,12.000000,2010.000000,745000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [92]:
# TODO:

# - investigate why there's no "Industrial, Agriculture, Residential Low Density Park" in the training data
# - try dropping rows BEFORE splitting the data
# - test 2 questions in notes (ie. separate models & weighted values) to get answers
training___data

,Id,LotFrontage,LotArea,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,YrSold,SaleType,SaleCondition,SalePrice,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,Street_Grvl,Street_Pave
1292,1293,60.0,6600,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,...,2009,WD,Normal,107500,0,0,0,1,0,1
1430,1431,60.0,21930,NaN,IR3,Lvl,AllPub,Inside,Gtl,Gilbert,...,2006,WD,Normal,192140,0,0,1,0,0,1
810,811,78.0,10140,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,2006,WD,Normal,181000,0,0,1,0,0,1
477,478,105.0,13693,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,2007,WD,Normal,380000,0,0,1,0,0,1
874,875,52.0,5720,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,...,2009,WD,Abnorml,66500,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,278,140.0,19138,NaN,Reg,Lvl,AllPub,Corner,Gtl,Gilbert,...,2010,WD,Normal,141000,0,0,1,0,0,1
763,764,82.0,9430,NaN,Reg,Lvl,AllPub,Inside,Gtl,NoRidge,...,2009,WD,Normal,337000,0,0,1,0,0,1
835,836,60.0,9600,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,...,2010,WD,Normal,128000,0,0,1,0,0,1
1216,1217,68.0,8930,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,...,2010,WD,Normal,112000,0,0,0,1,0,1
